In [252]:
#import libraries
from scipy import stats
import uproot
import matplotlib as mpl
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
from mpl_toolkits import mplot3d
from sklearn.cluster import DBSCAN
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import numpy as np

In [ ]:

#Creating the noise file 
def open_noise(filename) :
    noise0=uproot.open(filename) #using uproot to open the root file 
    noise1=noise0['pixTree']
    noise=pd.DataFrame(noise1['fData'].array(library="np"))
    noise=noise[noise.duplicated(subset=['row','col'])==True]
    noise.drop_duplicates(subset=['row','col'],inplace=True)  #one consider a pixel tobe noisy xhen on a short run (<1minute), with no source it declares a hit more than one time.
    X_noise=np.array(noise.col+((noise.chipId-4)*1024)) #changing the coordinates 
    Y_noise=np.array(noise.row)
    return X_noise,Y_noise

#put in the Noise data 
Noise1=open_noise('../Noise1')
Noise2=open_noise('../Noise2')
Noise3=open_noise('../Noise3')
Noise4=open_noise('../Noise4')


In [ ]:
#open the root file and getting the tree
def open_file (filename,id):
    file=uproot.open(filename)
    file1=file['pixTree']
    Data=file1['fData'].array(library="np")
    data=pd.DataFrame(Data)
    data['ldr']=id
    data.col=((data.chipId-4)*1024)+data.col
    return data

data=open_file("../Data1",1)
data2=open_file("../Data2",2)
data3=open_file("../Data3",3)
data4=open_file("../Data4",4)

data=pd.concat([data,data2])
data=pd.concat([data,data3])
data=pd.concat([data,data4])





fig, axes = plt.subplots(4)

display(data) #show the table with the different entries
axes[0].hist2d(data[data.ldr==1].col,data[data.ldr==1].row,bins=100,density=True) #plotting the data as a 2D histogram (at this point data from ladder 2 and 4 are flipped)
axes[1].hist2d(data[data.ldr==2].col,data[data.ldr==2].row,bins=100,density=True)
axes[2].hist2d(data[data.ldr==3].col,data[data.ldr==3].row,bins=100,density=True)
axes[3].hist2d(data[data.ldr==4].col,data[data.ldr==4].row,bins=100,density=True)
plt.show()

In [285]:
#remove noisy pixels 
def remove_noise (noise,data) :
        n_0=data[data.col.isin(noise[0]) & data.row.isin(noise[1])].index
        if len(n_0)>0:
            data.drop(index=n_0,inplace=True)
        return data 


data_filtered=remove_noise(Noise1,data[data.ldr==1])
data_filtered=pd.concat([data_filtered,remove_noise(Noise2,data[data.ldr==2])])
data_filtered=pd.concat([data_filtered,remove_noise(Noise2,data[data.ldr==3])])
data_filtered=pd.concat([data_filtered,remove_noise(Noise2,data[data.ldr==4])])


In [ ]:

#Displaying the filtered data (Monitoring the experiment)
display(data_filtered)
fig, axes = plt.subplots(4)
axes[0].hist2d(data_filtered[(data_filtered.ldr==1)&(data_filtered.chipId==8)].col,data_filtered[(data_filtered.ldr==1)&(data_filtered.chipId==8)].row,bins=[600,300],norm='symlog')
axes[1].hist2d(data_filtered[(data_filtered.ldr==2)&(data_filtered.chipId==8)].col,data_filtered[(data_filtered.ldr==2)&(data_filtered.chipId==8)].row,bins=[600,300],norm='symlog')
axes[2].hist2d(data_filtered[(data_filtered.ldr==3)&(data_filtered.chipId==8)].col,data_filtered[(data_filtered.ldr==3)&(data_filtered.chipId==8)].row,bins=[600,300],norm='symlog')
axes[3].hist2d(data_filtered[(data_filtered.ldr==4)&(data_filtered.chipId==8)].col,data_filtered[(data_filtered.ldr==4)&(data_filtered.chipId==8)].row,bins=[600,300],norm='symlog')
plt.show()

In [ ]:
#flipping the data from the even plane in y 
def flip_data_row (data) :
    data.row=512-data.row
    return data

data_filtered[(data_filtered.ldr==2)]=flip_data_row(data_filtered[(data_filtered.ldr==2)])
data_filtered[(data_filtered.ldr==4)]=flip_data_row(data_filtered[(data_filtered.ldr==4)])


In [ ]:
#refocusing on the used chipId
chipId=8
data_focused=data_filtered[data_filtered.chipId==chipId] 

In [ ]:
#Plotting superposed (to see Misalignements)
fig, axes = plt.subplots()
plt.hist2d(data_focused[data_focused.ldr==1].col,data_focused[data_focused.ldr==1].row,bins=[600,300],norm='symlog',cmap='Greens')
plt.hist2d(data_focused[data_focused.ldr==2].col,data_focused[data_focused.ldr==2].row,bins=[600,300],norm='symlog',cmap='Blues',alpha=0.2)
plt.hist2d(data_focused[data_focused.ldr==3].col,data_focused[data_focused.ldr==3].row,bins=[600,300],norm='symlog',cmap='Reds',alpha=0.2)
plt.hist2d(data_focused[data_focused.ldr==4].col,data_focused[data_focused.ldr==4].row,bins=[600,300],norm='symlog',cmap='viridis',alpha=0.2,cmin=1)
plt.axis('scaled')
plt.show()

In [294]:
#Clustering with DBSCAN  

def Clustering (data) :
    eps=1.5
    n=0
    Cluster=[]
    L=[]
    m=max(data.trgNum)
    for i in range(m+1) :#scan each event
        print('Doing clustering : ',(i*100)/m,' %',end='\r')
        if data[data.trgNum==i].empty ==False :#reove empty event
            if (i in L) ==False :
                L.append(i)
                data_trgNum= data[data.trgNum==i] 
                if len(data_trgNum)!=1 : 
                    X=data_trgNum[['row','col']].to_numpy()
                    clustering = DBSCAN(eps=eps,metric='euclidean',min_samples=1).fit(X) #computing the clusters : a cluster is a group of hits seperated by less than 1.5 pixels
                    labels = clustering.labels_ 
                    for i in labels :
                        Cluster.append(i+n)
                    n=max(labels)+1+n
                else :
                    Cluster.append(n)
                    n+=1
    return Cluster


 data_clusterized=do_clusters(data_focused)


In [ ]:

def do_clusters(data_focused): #we compute first the Clusters by ladder and them we put them all together in one DataFrame
    Clusters=[]
    for i in range (max(data_focused.ldr)):
        Clusters.append(Clustering(data_focused[data_focused.ldr==i+1]))
    #Setting the clusters to the right data
    data_clusterized=data_focused.copy()
    data_clusterized['cluster']=0
    for i in range(max(data_clusterized.ldr)):
        data_clusterized.loc[data_clusterized.ldr==i+1,'cluster']=np.array(Clusters[i])+max(data_clusterized.cluster)
    return data_clusterized

In [ ]:
#cluster size

def cluster_size (data) :
    Cluster_size=pd.DataFrame({'trgNum':[], 'ldr':[],'Cluster_size':[]})
    m=int(max(data.cluster))
    data= data.sort_values(by=['cluster'])
    for cluster in range(10000) :
        print('cluster_size_percentage : ',cluster/100,end='\r')
        Cluster_size=pd.concat([Cluster_size,pd.DataFrame({'trgNum':[data.loc[data.cluster==cluster,'trgNum'].values[0]],'ldr':[data.loc[data.cluster==cluster,'ldr'].values[0]],'Cluster_size':[len(data[data.cluster==cluster])]})])
    return(Cluster_size)

Cluster_size=cluster_size(data_clusterized)


In [ ]:
#displaying the distribution of the cluster size
Cluster_size['Cluster_size'].hist(bins=20)
plt.title('Cluster size distribution')
plt.ylabel('Entries')
plt.xlabel('Cluster size')

In [ ]:
#Cluster by trgNum 
Cluster_by_trgNum=[len(Cluster_size[Cluster_size.trgNum==t]) for t in range (1,int(max(Cluster_size.trgNum))+1)]
plt.hist(Cluster_by_trgNum,bins=10)
plt.title('Cluster by trigger Number')
plt.ylabel('Entries')
plt.xlable('Number of cluster by trigger number')

In [296]:

#computing barycenter
def reduce_by_barycenter_new (data) :
    data_sorted =data.sort_values(by=['cluster'])
    data_reduced=pd.DataFrame({'trgNum':[],'ldr':[],'cluster':[],'Xbar':[],'Ybar':[]})
    m=max(data_sorted.cluster)
    X=[]
    Y=[]
    n=0
    for index, entry in data_sorted.iterrows() :
        print('Reduction : ',(n*100/m)," %",end='\r')
        if entry.cluster == n :
            X.append(entry.col)
            Y.append(entry.row)
        else :
            e=pd.DataFrame({'trgNum':[entry.trgNum],'ldr':[entry.ldr],'cluster':[entry.cluster],'Xbar':[np.mean(X)],'Ybar':[np.mean(Y)]})
            data_reduced=pd.concat([data_reduced,e])
            X=[]
            Y=[]
            X.append(entry.col)
            Y.append(entry.row)
            n+=1
    return data_reduced


data_reduced=reduce_by_barycenter_new (data_clusterized)



In [ ]:
 #Efficiency (4 ladders hit for a trigger)
n=0
for t in range (10000):
    data_trg=data_reduced[data_reduced.trgNum==t]
    if len(data_trg[data_trg.ldr==1])>0 and len(data_trg[data_trg.ldr==2])>0 and len(data_trg[data_trg.ldr==3])>0 and len(data_trg[data_trg.ldr==4])>0 :
        n+=1
print('Efficiency : ',n/100)

In [297]:
#fit the data 

def chi2 (xexp,xfit): #computing chi2
    chi2 =0 
    for i in range (len(xexp)):
        chi2+=((xexp[i]-xfit[i])**2)/xexp[i]
    return chi2

def tracking (X,Y,Z0,t) :
    slope_X, intercept_X, r_value, p_value, std_err= stats.linregress(Z0, X)
    slope_Y, intercept_Y, r_value, p_value, std_err= stats.linregress(Z0, Y)
    Mean_Residuals_X=np.sqrt(np.mean((slope_X*np.array(Z0)+intercept_X-np.array(X))**2))#quadratique
    Mean_Residuals_Y=np.sqrt(np.mean((slope_Y*np.array(Z0)+intercept_Y-np.array(Y))**2))#quadratique
    return pd.DataFrame({'trgNum':[t],'Mean_Residuals_X':[Mean_Residuals_X],'Mean_Residuals_Y':[Mean_Residuals_Y],'slope_X':[slope_X],'intercept_X':[intercept_X],'slope_Y':[slope_Y],'intercept_Y':[intercept_Y]})


In [ ]:

#Alignement from the tracking : In this more precise alignement technique we compute all the residuals from a track (3 points fitting, distance from the track of the 4th one)

def precise_alignement (data,Z0) :
    Distances_X=[[],[],[],[]]
    Distances_Y=[[],[],[],[]]
    m=int(max(data.ldr))
    for t in range(10000) :
        data_trgNum=data[data.trgNum==t]
        print ("precise_alignement_percentage : ",t/100, end='\r')
        if (len(data_trgNum)==m and data_trgNum.ldr.nunique() == data_trgNum.ldr.size):
            for k in range(1,m):
                X=[]
                Y=[]
                Z=[]
                for j in range (m):
                    #if j!=k:
                    X.append(data_trgNum.loc[data_trgNum.ldr==j+1,'Xbar'].values[0])
                    Y.append(data_trgNum.loc[data_trgNum.ldr==j+1,'Ybar'].values[0])
                    Z.append(Z0[j])
                T=tracking (X,Y,Z,t)
                distance_x=T.at[0,'slope_X']*Z0[k]+T.at[0,'intercept_X']-data_trgNum.loc[data_trgNum.ldr==k+1,'Xbar'].values[0]
                distance_y=T.at[0,'slope_Y']*Z0[k]+T.at[0,'intercept_Y']-data_trgNum.loc[data_trgNum.ldr==k+1,'Ybar'].values[0]
                Distances_X[k].append(distance_x)
                Distances_Y[k].append(distance_y)
    return pd.DataFrame({'dx':Distances_X,'dy':Distances_Y})




In [ ]:
#Z corresponds to the original coordinates
Z=[0,10.6*10000/30,145*10000/30,155.6*10000/30]
Distances=precise_alignement(data_reduced,Z)
print('x',np.mean(Distances['dx'][0]),np.mean(Distances['dx'][1]),np.mean(Distances['dx'][2]),np.mean(Distances['dx'][3]),'Y',np.mean(Distances['dy'][0]),np.mean(Distances['dy'][1]),np.mean(Distances['dy'][2]),np.mean(Distances['dy'][3]))

In [ ]:
#moving from the correction that we had previously
def move (data,X,Y):
    for i in range(1,int(max(data.ldr))) :
        data.loc[data.ldr==i+1,'Xbar']=data[data.ldr==i+1].Xbar+np.mean(X[i])
        data.loc[data.ldr==i+1,'Ybar']=data[data.ldr==i+1].Ybar+np.mean(Y[i])
    return data


data_aligned=data_reduced.copy()
data_aligned=move(data_aligned,Distances['dx'],Distances['dy'])



In [233]:
#computing all the tracks for a run  (we do it on 10000 to avoid long running time)

def make_tracks (data,Z) :
    Tracks=pd.DataFrame({'trgNum':[],'Mean_Residuals_X':[],'Mean_Residuals_Y':[],'slope_X':[],'intercept_X':[],'slope_Y':[],'intercept_Y':[]})
    m=int(max(data.ldr))
    data= data.sort_values(by=['trgNum'])
    for t in range(10000) :
        print ("make_tracks_percentage : ",t/100, end='\r')
        data_trgNum=data[data.trgNum==t]
        if (len(data_trgNum)==m and data_trgNum.ldr.nunique() == data_trgNum.ldr.size):
            X=[data_trgNum.loc[data_trgNum.ldr==i+1,'Xbar'].values[0] for i in range (m)]
            Y=[data_trgNum.loc[data_trgNum.ldr==i+1,'Ybar'].values[0] for i in range (m)]
            T=tracking (X,Y,Z,t)
            Tracks=pd.concat([Tracks,T])
    return Tracks

In [ ]:
Tracks_aligned=make_tracks(data_aligned,Z)

In [ ]:
#Looping on alignement
DATA=[data_aligned]
Tracks=[Tracks_aligned]
for i in range (100) :
    data=DATA[i].copy()
    Distances=precise_alignement(data,Z)
    DATA.append(move(data,Distances['dx'],Distances['dy']))
    Tracks.append(make_tracks(data,Z))

#displaying the evolution of residuals according to the iteration 
Residual=[]
for i in range(len(Tracks)):
    Residual.append(np.mean(Tracks[i].Mean_Residuals_Y))
plt.scatter(np.arange(0,101,1),Residual)

In [ ]:
#event display (t: trigger number, absi: Z coordinate, ordo: X or Y coordinate)

def event_display(data,track,t,absi,ordo :str,label) :
    Absi=np.array(absi)
    plt.title("event display of " +ordo +" in function of Z")
    plt.scatter(Absi,data[data.trgNum==t][ordo+'bar'])
    plt.plot(Absi,track.loc[track.trgNum==t,'slope_'+ordo].values[0]*Absi+track.loc[track.trgNum==t,'intercept_'+ordo].values[0],label=label)
    

event_display(data_aligned,Tracks_aligned,2000,Z,'X','aligned')
plt.legend()
plt.show()


In [ ]:
#mean event display (the average event)

def mean_event_display(data,track,Z,ordo:str,label) :
    Z=np.array(Z)
    plt.scatter(Z,[np.mean(data.loc[data['trgNum'].isin(np.array(track.trgNum))&(data.ldr==i+1),ordo+'bar'])for i in range(max(data.ldr))])
    plt.plot(Z,np.mean(track['slope_'+ordo])*Z+np.mean(track['intercept_'+ordo]),label=label)
    plt.title(ordo+'(Z) in average, points: real, line: reconstructed track',fontsize=16)

In [ ]:
#3D mean event (no rotation)


def mean_event_3d (data,Track):
    fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

    # Make data.
    X = np.arange(4096, 5120, 1)
    Y = np.arange(0, 512, 1)
    a=len(X)
    X, Y = np.meshgrid(X, Y)
    #Z could me changed according to a plane equation 
    Z0=np.zeros((len(Y),a))
    Z=np.ones((len(Y),a))
    Z1=Z*10.6*10000/30
    Z2=Z*145*10000/30
    Z3=Z*155.6*10000/30
    # Plot the ladders
    ax.plot_surface(X, Y, Z0,color='Grey',alpha=0.2)
    ax.plot_surface(X, Y, Z1,color='Grey',alpha=0.2)
    ax.plot_surface(X, Y, Z2,color='Grey',alpha=0.2)
    ax.plot_surface(X, Y, Z3,color='Grey',alpha=0.2)
    # plot the measured points
    ax.scatter([np.mean(data.loc[data['trgNum'].isin(np.array(Track.trgNum))&(data.ldr==i+1),'Xbar'])for i in range(int(max(data.ldr)))],[np.mean(data.loc[data['trgNum'].isin(np.array(Track.trgNum))&(data_reduced.ldr==i+1),'Ybar'])for i in range(int(max(data.ldr)))],[1000,10.6*10000/30+1,145*10000/30+1,155.6*10000/30+1],color='r',marker='+',s=200)
    #plot the reconstructed track
    ax.plot(np.mean(Track['slope_X'])*np.array([1000,10.6*10000/30+1,145*10000/30+1,155.6*10000/30+1])+np.mean(Track['intercept_X']), np.mean(Track['slope_Y'])*np.array([1000,10.6*10000/30+1,145*10000/30+1,155.6*10000/30+1])+np.mean(Track['intercept_Y']),[1000,10.6*10000/30+1,145*10000/30+1,155.6*10000/30+1] ,color='r')
    
    #customization
    ax.view_init(vertical_axis='y',azim=-45)
    ax.set_zlim(0,55000)
    ax.tick_params( labelsize = 8)
    ax.set_ylim3d(0, 1024)
    ax.set_xlim3d(4096, 5120)
    ax.set_xlabel("Horizontal axis [pixels]",fontsize=12)
    ax.set_ylabel("Vertical axis [pixels]",fontsize=12)
    ax.set_zlabel("Beam axis [pixels]",fontsize=12)
    plt.title("Overview of the mean event, zoomed on the chip of interest",fontsize=16)

mean_event_3d(data_aligned,Tracks_aligned)

In [ ]:
#Defining the plane of the ladder with an equation : a_x*x+a_y*y+a_z*z+cte=0
def plan_echelle(theta,psi,phi,x,y,z) :
    phiy,phiz,thetaz,thetax,psiy,psix=1,1,1,1,1,1
    if theta== 0 : 
        psix,thetax=0,0
    if phi== 0 : 
         phiy,psiy=0,0
    if (phi!=0 and psi!=0) or (theta!=0 and psi!=0):
        psix=np.sin(psi)
        psiy=-np.cos(psi)
        thetax=1
        phiy=1
    else :
        psiy=1
        psix=1
    if phi!=0 :
        phiz=-np.cos(phi)
        phiy=np.sin(phi)
    if theta!=0 :
        thetaz=-np.cos(theta)
        thetax=np.sin(theta)
    a_x=psix*thetax
    a_y=psiy*phiy
    a_z=thetaz*phiz
    cte=-(a_x*x+a_y*y+a_z*z)
    return (pd.DataFrame({'a_x':[a_x],'a_y':[a_y],'a_z':[a_z],'cte':[cte]}))

def intersect(t_x,t_y,c_x,c_y,P): #define the intersection of a track with a given plane P
    z_inter=(-P.cte[0]-P.a_y[0]*c_y-P.a_x[0]*c_x)/(P.a_z[0]+P.a_y[0]*t_y+P.a_x[0]*t_x)
    return z_inter



In [ ]:
#computing the tracks for a given ladder and for a given ladder and a given event

def make_tracks_ladder (data,ldr) : 
    Tracks=pd.DataFrame({'trgNum':[],'Mean_Residuals_X':[],'Mean_Residuals_Y':[],'slope_X':[],'intercept_X':[],'slope_Y':[],'intercept_Y':[],'ldr':[],'Residual_X_'+str(ldr['id'][0]):[],'Residual_Y_'+str(ldr['id'][0]):[],'Residual_Z_'+str(ldr['id'][0]):[],'z_inter':[],'Residual_tot'+str(ldr['id'][0]):[]})
    m=int(max(data.ldr))
    data= data.sort_values(by=['trgNum'])
    for t in range(10000) :
        print ("make_tracks_percentage : ",t/100, end='\r')
        data_trgNum=data[data.trgNum==t]
        if (len(data_trgNum)==m and data_trgNum['ldr'].nunique() == data_trgNum['ldr'].size):
            
            X=[data_trgNum.loc[data_trgNum.ldr==i+1,'Xbar'].values[0] for i in range (m)]
            Y=[data_trgNum.loc[data_trgNum.ldr==i+1,'Ybar'].values[0] for i in range (m)]
            Z=[data_trgNum.loc[data_trgNum.ldr==i+1,'Z'].values[0] for i in range (m)] 
            
            T=tracking (X,Y,Z,t)
            T['ldr']=ldr['id']
            P=plan_echelle(ldr.theta[0],ldr.psi[0],ldr.phi[0],ldr.x[0],ldr.y[0],ldr.z[0])
            z_inter=intersect(T['slope_X'].values[0],T['slope_Y'].values[0],T['intercept_X'].values[0],T['intercept_Y'].values[0],P)
            #we compute only the residual for the moved ladder.
            T['Residual_X_'+str(ldr['id'][0])]=T['slope_X'].values[0]*z_inter+ T['intercept_X'].values[0] -X[ldr['id'][0]-1]
            T['Residual_Y_'+str(ldr['id'][0])]=-T['slope_Y'].values[0]*z_inter+T['intercept_Y'].values[0]-Y[ldr['id'][0]-1]
            T['Residual_Z_'+str(ldr['id'][0])]=z_inter-Z[ldr['id'][0]-1]
            T['Residual_tot'+str(ldr['id'][0])]=np.sqrt(T['Residual_Z_'+str(ldr['id'][0])]**2+T['Residual_Y_'+str(ldr['id'][0])]**2+T['Residual_X_'+str(ldr['id'][0])]**2)
            T['z_inter']=z_inter
            Tracks=pd.concat([Tracks,T])
            
    return Tracks

def make_tracks_ladder_event (data,ldr:,t:int) : # ldr is the ladder number and t the trigger number 
    m=int(max(data.ldr))
    data_trgNum=data[data.trgNum==t]   
    if (len(data_trgNum)==m and data_trgNum['ldr'].nunique() == data_trgNum['ldr'].size):
        X=[data_trgNum.loc[data_trgNum.ldr==i+1,'Xbar'].values[0] for i in range (m)]
        Y=[data_trgNum.loc[data_trgNum.ldr==i+1,'Ybar'].values[0] for i in range (m)]
        Z=[data_trgNum.loc[data_trgNum.ldr==i+1,'Z'].values[0] for i in range (m)] 
        T=tracking (X,Y,Z,t)
        T['ldr']=ldr['id']
        P=plan_echelle(ldr.theta[0],ldr.psi[0],ldr.phi[0],ldr.x[0],ldr.y[0],ldr.z[0])
        z_inter=intersect(T['slope_X'].values[0],T['slope_Y'].values[0],T['intercept_X'].values[0],T['intercept_Y'].values[0],P)
        #we compute only the residual for the moved ladder.
        T['Residual_X_'+str(ldr['id'][0])]=T['slope_X'].values[0]*z_inter+ T['intercept_X'].values[0] -X[ldr['id'][0]-1]
        T['Residual_Y_'+str(ldr['id'][0])]=-T['slope_Y'].values[0]*z_inter+T['intercept_Y'].values[0]-Y[ldr['id'][0]-1]
        T['Residual_Z_'+str(ldr['id'][0])]=z_inter-Z[ldr['id'][0]-1]
        T['Residual_tot'+str(ldr['id'][0])]=np.sqrt(T['Residual_Z_'+str(ldr['id'][0])]**2+T['Residual_Y_'+str(ldr['id'][0])]**2+T['Residual_X_'+str(ldr['id'][0])]**2)
        T['z_inter']=z_inter            
    return T

In [ ]:
#Computing the global and local derivatives

def global_derivative_residual_event (data,dpsi,dtheta,dphi,dz,dx,dy,L,Track_init,t) : #inputs : movements that we want to try, Track_init=> all the inital tracks, L the ladder that you are working on
    track0=Track_init[Track_init.trgNum==t]
    data_moved=data.copy()
    L_moved=pd.DataFrame({'id':[L.id[0]],'theta':[L.theta[0]+dtheta],'phi':[L.phi[0]+dphi],'psi':[L.psi[0]+dpsi],'x':[L.x[0]+dx],'y':[L.y[0]+dy],'z':[L.z[0]+dz]})
    ldr=L['id'].values[0]

    #Changing the data according to the given rotations and translations
    if dtheta!=0 :
        Norm1=np.sqrt((data_moved.loc[data_moved.ldr==ldr,'Xbar']**2)+((data_moved.loc[data_moved.ldr==ldr,'Z']-L.z[0])**2))
        data_moved.loc[data_moved.ldr==ldr,'Xbar']=Norm1*np.cos(L_moved.theta[0])
        data_moved.loc[data_moved.ldr==ldr,'Z']=Norm1*np.sin(L_moved.theta[0])+L.z[0]
    if dphi!=0 :
        Norm2=np.sqrt((data_moved.loc[data_moved.ldr==ldr,'Ybar']**2)+((data_moved.loc[data_moved.ldr==ldr,'Z']-L.z[0])**2))
        data_moved.loc[data_moved.ldr==ldr,'Ybar']=Norm2*np.cos(L_moved.phi[0])
        data_moved.loc[data_moved.ldr==ldr,'Z']=Norm2*np.sin(L_moved.phi[0])+L.z[0]
    if dpsi!=0 :
        Norm3=np.sqrt((data_moved.loc[data_moved.ldr==ldr,'Xbar']**2)+((data_moved.loc[data_moved.ldr==ldr,'Ybar'])**2))
        data_moved.loc[data_moved.ldr==ldr,'Xbar']=data_moved.loc[data_moved.ldr==ldr,'Xbar']*np.cos(L_moved.psi[0])-data_moved.loc[data_moved.ldr==ldr,'Ybar']*np.sin(L_moved.psi[0])
        data_moved.loc[data_moved.ldr==ldr,'Ybar']=data_moved.loc[data_moved.ldr==ldr,'Xbar']*np.sin(L_moved.psi[0])+data_moved.loc[data_moved.ldr==ldr,'Ybar']*np.cos(L_moved.psi[0])
    data_moved.loc[data_moved.ldr==ldr,'Xbar']+=dx
    data_moved.loc[data_moved.ldr==ldr,'Ybar']+=dy
    data_moved.loc[data_moved.ldr==ldr,'Z']+=dz

    #Compute the new track
    track1=make_tracks_ladder_event(data_moved,L_moved,t)
    Difference=track1['Residual_tot'+str(ldr)].values[0]-track0['Residual_tot'+str(ldr)].values[0]
    return Difference,track1,track0, data_moved,L_moved #We deliver not really the derivative put the new difference (f(x+dx)-f(x)) we will divide by dx to get an approximation of the derivative

def local_derivative_event (data,dx,dy,tx,ty,L,Track_init,t) : 
    track0=Track_init[Track_init.trgNum==t]
    track1=track0.copy()
    track1['slope_X']=track1['slope_X']+tx
    track1['intercept_X']=track1['intercept_X']+dx
    track1['slope_Y']=track1['slope_Y']+ty
    track1['intercept_Y']=track1['intercept_Y']+dy
    m=int(max(data.ldr))
    data_trgNum=data[data.trgNum==t]
    ldr=L['id'].values[0]
    if (len(data_trgNum)==m and data_trgNum['ldr'].nunique() == data_trgNum['ldr'].size):
        X=[data_trgNum.loc[data_trgNum.ldr==i+1,'Xbar'].values[0] for i in range (m)]
        Y=[data_trgNum.loc[data_trgNum.ldr==i+1,'Ybar'].values[0] for i in range (m)]
        Z=[data_trgNum.loc[data_trgNum.ldr==i+1,'Z'].values[0] for i in range (m)] 
        P=plan_echelle(L.theta[0],L.psi[0],L.phi[0],L.x[0],L.y[0],L.z[0])
        z_inter=intersect(track1['slope_X'].values[0],track1['slope_Y'].values[0],track1['intercept_X'].values[0],track1['intercept_Y'].values[0],P)
        #we compute only the residual for the moved ladder.
        track1['Residual_X_'+str(L['id'][0])]=track1['slope_X'].values[0]*z_inter+ track1['intercept_X'].values[0] -X[ldr-1]
        track1['Residual_Y_'+str(L['id'][0])]=-track1['slope_Y'].values[0]*z_inter+track1['intercept_Y'].values[0]-Y[ldr-1]
        track1['Residual_Z_'+str(L['id'][0])]=z_inter-Z[ldr-1]
        track1['Residual_tot'+str(L['id'][0])]=np.sqrt(track1['Residual_Z_'+str(L['id'][0])]**2+track1['Residual_Y_'+str(L['id'][0])]**2+track1['Residual_X_'+str(L['id'][0])]**2)
        track1['z_inter']=z_inter 
    Difference=track1['Residual_tot'+str(ldr)].values[0]-track0['Residual_tot'+str(ldr)].values[0]
    return Difference,track1,track0, data,L #We deliver not really the derivative put the new difference (f(x+dx)-f(x)) we will divide by dx to get an approximation of the derivative

In [ ]:

#making the CSV of derivatives and Residual for mille (only 1000 tracks)

#labels : psi=50,theta=60,phi=70,z=80,x=90,y=100 and the unity number is the ladder number
import csv 
def make_mille(data,name:str) :
    Label=np.array([50,60,70,80,90,100])
    d=0.000001
    dos=open(name+'.csv','w',newline='')
    writer=csv.writer(dos)
    writer.writerow(['Iteration','NLC', 'derLc','NGL', 'derGl', 'label','rMeas', 'sigma'])
    Ladders=[]
    T=[]
    for q in range(2,5): #Checking each ladder
        print('Running on ladder : ',q)
        Ladders.append(pd.DataFrame({'id':[q],'theta':[0],'phi':[0],'psi':[0],'x':[0],'y':[0],'z':[int(Z[q-1]*10000/30)]})) #Z is the initial distance at which the origin of the plane is
        T.append(make_tracks_ladder (data,Ladders[q-2]))
        t0=0
        #Checking each event useful
    for t in T[1].trgNum : #looping on the different events
            t0+=1
            if t0>1000: #keeping only the 1000 first ones (this is enough to compute millepede)
                break
            Data=[]
            for l in range(2) : #Compute residual in X and then residual in Y 
                for i in range(2,5) :
                    Labels=Label+i

                    print('Running on ladder : ',i,', percentage : ', t0, end='\r')
                    Global=[]
                    Local=[]
                    j=0
                        #Compute global derivatives
                    while j <6:#looping on the different rotations and translations
                        G=[0]*6
                        G[j]=d
                        psi,theta,phi,z,x,y=G[0],G[1],G[2],G[3],G[4],G[5]
                        A=global_derivative_residual_event(data,psi,theta,phi,z,x,y,Ladders[i-2],T[i-2],t) 
                        a=A[1]['Residual_tot'+str(i)].values[0]-A[2]['Residual_tot'+str(i)].values[0]
                        B=global_derivative_residual_event(data,-psi,-theta,-phi,-z,-x,-y,Ladders[i-2],T[i-2],t)
                        b=B[1]['Residual_tot'+str(i)].values[0]-B[2]['Residual_tot'+str(i)].values[0]
                        if (abs((a/d)-(b/(-d)))<0.01) : #We check that tha we are in a vincinity of the derivate in 0
                            Global.append(a/d) #derivate evaluate in 0 (approximation)
                            j+=1
                            d=0.000001
                        else :
                            d=d/10
                    #Compute local derivatives
                    k=0
                    d=0.00001
                    while k<2 :#looping btw the interesect and the slope
                        L=[0]*2
                        L[k]=d
                        dx,dy,tx,ty=(l-1)*L[0],(l-1)*L[1],l*L[0],l*L[1]
                        C=local_derivative_event (data,dx,dy,tx,ty,Ladders[i-2],T[i-2],t)
                        c=C[1]['Residual_tot'+str(i)].values[0]-C[2]['Residual_tot'+str(i)].values[0]
                        E=local_derivative_event (data,-dx,-dy,-tx,-ty,Ladders[i-2],T[i-2],t)
                        e=E[1]['Residual_tot'+str(i)].values[0]-E[2]['Residual_tot'+str(i)].values[0]
                        if (abs((c/d)-(e/(-d)))<0.1) :
                            Local.append(c/d) #derivate evaluate in 0 (approximation)
                            k+=1
                            d=0.00001
                        else :
                            d=d/10
                    Local=np.array(Local)
                    Global=np.array(Global)
                    if l==0 :
                        Data.append([t,4, str(Local),2, str(Global), str(Labels),T[i-2].loc[T[i-2].trgNum==t,'Residual_X_'+str(i)].values[0], 1])
                    else :
                        Data.append([t,4, str(Local),2, str(Global), str(Labels),T[i-2].loc[T[i-2].trgNum==t,'Residual_Y_'+str(i)].values[0], 1])
            for row in Data: #just to put the data in a right format
                    row = [cell.replace('\n', ' ') if isinstance(cell, str) else cell for cell in row]
                    print(row)
                    writer.writerow(row)

In [ ]:
#Be sure to create a Z column in your data before running make mille